<a href="https://colab.research.google.com/github/YimingXu1/DS-notes/blob/main/Projects/U.S.%2520Patent%2520Phrase%2520to%2520Phrase%2520Matching/Bert_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is a stencil for bert model training

# Directory Setting

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip3 install datasets
!pip3 install transformers
!pip3 install sentencepiece
!pip3 install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
import shutil

import torch
from torch.utils.data import DataLoader, Dataset
import datasets, transformers
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

os.environ["WANDB_DISABLED"] = "true"

# CFG

In [4]:
class CFG:
    input_path = './drive/MyDrive/Kaggle/USPatent/uspatent/'
    #model_path = './drive/MyDrive/Kaggle/deberta-v3-small/'
    model_path = 'microsoft/deberta-v3-large' #'./drive/MyDrive/Kaggle/USPatent/Models/'
    
    learning_rate = 2e-5
    weight_decay = 0.01
    num_fold = 5
    epochs = 5
    batch_size =1
    max_input_length = 130
    batch_size = 64
    num_workers = 2



# Preprocessing

In [5]:
# https://www.kaggle.com/code/abhishek/phrase-matching-folds
def create_folds(data, num_splits):
    # we create a new column called kfold and fill it with -1
    data["fold"] = -1
    
    # the next step is to randomize the rows of the data
    # data = data.sample(frac=1).reset_index(drop=True)

    # calculate number of bins by Sturge's rule
    # I take the floor of the value, you can also
    # just round it
    # num_bins = int(np.floor(1 + np.log2(len(data))))
    
    # bin targets
    data.loc[:, "bins"] = pd.cut(
        data["score"], bins=5, labels=False
    )
    
    # initiate the kfold class from model_selection module
    kf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)
    
    # fill the new kfold column
    # note that, instead of targets, we use bins!
    for f, (t_, v_) in enumerate(kf.split(X=data, y=data.bins.values)):
        data.loc[v_, 'fold'] = f
    
    # drop the bins column
    data = data.drop("bins", axis=1)

    # return dataframe with folds
    return data

In [6]:
train_df = pd.read_csv(f"{CFG.input_path}train.csv")
titles = pd.read_csv(f"{CFG.input_path}titles.csv")
train_df = train_df.merge(titles, left_on='context', right_on='code')
# ====================================================
# CPC Data
# ====================================================
cpc_texts = torch.load("./drive/MyDrive/Kaggle/folds-dump-the-two-paths-fix/cpc_texts.pth")
train_df['context_text'] = train_df['context'].map(cpc_texts)
train_df['text'] = train_df['anchor'] + '[SEP]' + train_df['target'] + '[SEP]'  + train_df['context_text']
train_df['text'] = train_df['text'].apply(str.lower)
display(train_df.head())

,id,anchor,target,context,score,code,title,section,class,subclass,group,main_group,context_text,text
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]abatement of pollution[sep]human...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]act of abating[sep]human necessi...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]active catalyst[sep]human necess...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]eliminating process[sep]human ne...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,A47,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,A,47.0,NaN,NaN,NaN,HUMAN NECESSITIES. FURNITURE; DOMESTIC ARTICLE...,abatement[sep]forest region[sep]human necessit...


In [7]:
train_df = create_folds(train_df, CFG.num_fold)

In [8]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Dataset

In [9]:
class TrainDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['text'].values.astype(str)
        self.targets = df['target'].values.astype(str)
        self.label = df['score'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        label = self.label[item]
        
        
        inputs = tokenizer(inputs, 
                    targets,
                    max_length=64,
                    padding='max_length',
                    truncation=True )
        return {**inputs,
               'labels':torch.as_tensor(label, dtype=torch.float) }

# Fine-tuning the model

In [10]:
from datasets import load_metric
metric = load_metric('glue', 'stsb')
metric

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)

In [11]:
oof_df = pd.DataFrame()
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)

metric_name = "pearson"
model_name = CFG.model_path.split("/")[-1]
    
args = TrainingArguments(
    output_dir=f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=CFG.learning_rate,
    per_device_train_batch_size=CFG.batch_size,
    per_device_eval_batch_size=CFG.batch_size,
    num_train_epochs=CFG.epochs,
    weight_decay=CFG.weight_decay,  
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    save_total_limit=1
)
    
#for fold in range(1):

fold = 0
    
tr_data = train_df[train_df['fold']!=fold].reset_index(drop=True)
va_data = train_df[train_df['fold']==fold].reset_index(drop=True)
tr_dataset = TrainDataset(tr_data)
va_dataset = TrainDataset(va_data)
    
trainer = Trainer(
    model,
    args,
    train_dataset=tr_dataset,
    eval_dataset=va_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
    
trainer.train()
shutil.rmtree(f"{model_name}-finetuned")
trainer.save_model(f"{model_name}-finetuned-{fold}")
    
outputs = trainer.predict(va_dataset)
predictions = outputs.predictions.reshape(-1)
va_data['preds'] = predictions
oof_df = pd.concat([oof_df, va_data])

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

RuntimeError: ignored

In [ ]:
fold = 1
    
tr_data = train_df[train_df['fold']!=fold].reset_index(drop=True)
va_data = train_df[train_df['fold']==fold].reset_index(drop=True)
tr_dataset = TrainDataset(tr_data)
va_dataset = TrainDataset(va_data)
    
trainer = Trainer(
    model,
    args,
    train_dataset=tr_dataset,
    eval_dataset=va_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
    
trainer.train()
shutil.rmtree(f"{model_name}-finetuned")
trainer.save_model(f"{model_name}-finetuned-{fold}")
    
outputs = trainer.predict(va_dataset)
predictions = outputs.predictions.reshape(-1)
va_data['preds'] = predictions
oof_df = pd.concat([oof_df, va_data])

In [ ]:
predictions = oof_df['preds'].values
label = oof_df['score'].values
eval_pred = predictions, label
compute_metrics(eval_pred)

In [11]:
metric_name = "pearson"
model_name = CFG.model_path.split("/")[-1]
batch_size = 8
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    save_total_limit=1,
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
train_dataset = TrainDataset(train_df[train_df['fold'] != 0])
val_dataset = TrainDataset(train_df[train_df['fold'] == 0])
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 29178
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 18240


Epoch,Training Loss,Validation Loss,Pearson,Spearmanr
1,0.068900,0.067497,0.009827,0.011585
2,0.068600,0.066796,-0.010143,-0.008760
3,0.067200,0.066925,0.011473,0.005749


***** Running Evaluation *****
  Num examples = 7295
  Batch size = 8
Saving model checkpoint to deberta-v3-large-finetuned/checkpoint-3648
Configuration saved in deberta-v3-large-finetuned/checkpoint-3648/config.json
Model weights saved in deberta-v3-large-finetuned/checkpoint-3648/pytorch_model.bin
tokenizer config file saved in deberta-v3-large-finetuned/checkpoint-3648/tokenizer_config.json
Special tokens file saved in deberta-v3-large-finetuned/checkpoint-3648/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 7295
  Batch size = 8
Saving model checkpoint to deberta-v3-large-finetuned/checkpoint-7296
Configuration saved in deberta-v3-large-finetuned/checkpoint-7296/config.json
Model weights saved in deberta-v3-large-finetuned/checkpoint-7296/pytorch_model.bin
tokenizer config file saved in deberta-v3-large-finetuned/checkpoint-7296/tokenizer_config.json
Special tokens file saved in deberta-v3-large-finetuned/checkpoint-7296/special_tokens_map.json
***** Runnin

In [ ]:
oof_df = pd.DataFrame()

trainer.save_model(f"{model_name}-finetuned-{fold}")

outputs = trainer.predict(va_dataset)
predictions = outputs.predictions.reshape(-1)
val_dataset['preds'] = predictions
oof_df = pd.concat([oof_df, va_data])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
train_dataset = TrainDataset(train_df[train_df['fold'] != 1])
val_dataset = TrainDataset(train_df[train_df['fold'] == 1])
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

trainer.train()

trainer.save_model(f"{model_name}-finetuned-{fold}")

outputs = trainer.predict(va_dataset)
predictions = outputs.predictions.reshape(-1)
val_dataset['preds'] = predictions
oof_df = pd.concat([oof_df, va_data])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
train_dataset = TrainDataset(train_df[train_df['fold'] != 2])
val_dataset = TrainDataset(train_df[train_df['fold'] == 2])
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

trainer.train()

trainer.save_model(f"{model_name}-finetuned-{fold}")

outputs = trainer.predict(va_dataset)
predictions = outputs.predictions.reshape(-1)
val_dataset['preds'] = predictions
oof_df = pd.concat([oof_df, va_data])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
train_dataset = TrainDataset(train_df[train_df['fold'] != 3])
val_dataset = TrainDataset(train_df[train_df['fold'] == 3])
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

trainer.train()

trainer.save_model(f"{model_name}-finetuned-{fold}")

outputs = trainer.predict(va_dataset)
predictions = outputs.predictions.reshape(-1)
val_dataset['preds'] = predictions
oof_df = pd.concat([oof_df, va_data])

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(CFG.model_path, num_labels=1)
train_dataset = TrainDataset(train_df[train_df['fold'] != 4])
val_dataset = TrainDataset(train_df[train_df['fold'] == 4])
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

trainer.train()

trainer.save_model(f"{model_name}-finetuned-{fold}")

outputs = trainer.predict(va_dataset)
predictions = outputs.predictions.reshape(-1)
val_dataset['preds'] = predictions
oof_df = pd.concat([oof_df, va_data])